### -------------Yanolja crawling---------------

In [93]:
import scrapy, requests
from scrapy.http import TextResponse
import pandas as pd
import json

In [3]:
# 1. 프로젝트 생성

In [44]:
!rm -rf yanolja_hotel
!scrapy startproject yanolja_hotel

New Scrapy project 'yanolja_hotel', using template directory '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /home/ubuntu/python3/notebook/project/crawling/crawling-repo-2/in_progress/yanolja_hotel

You can start your first spider with:
    cd yanolja_hotel
    scrapy genspider example example.com


In [6]:
# 2. xpath 찾기 : headers 설정

In [ ]:
#호텔 key값 뽑아내기

In [95]:
import requests #호텔 key값 얻는 모듈 생성 -> yanolja.py
import datetime

def get_hotel_keys():    
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36", 
           "X-Requested-With": "XMLHttpRequest"
              }
    today = datetime.date.today()
    checkin_date = today.strftime("%Y-%m-%d")
    checkout_date = (today + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
    region = "서울"
    num=0
    page = 1
    key_info = []
    while True:
        parent_url = f"https://www.yanolja.com/api/v1/contents/search?advert=KEYWORD&capacityAdult=2&capacityChild=0\
    &checkinDate={checkin_date}&checkoutDate={checkout_date}&hotel=1&keyword=서울&lat=37.50681&lng=127.06624&page={page}&limit=20&\
    searchKeyword={region}&searchType=hotel&sort=100&pathDivision=keyword-hotel&themes="
        req = requests.get(parent_url, headers=headers)
        response = req.json()
        #print('page=', str(apage))
        page += 1

        if response["motels"]["counts"] > 0:
            key = response["motels"]
            key_info.append(key)
        else:
            break
    hotel_keys = []
    for i in key_info:
        for j in i['lists']:
            hotel_keys.append(j['key'])        
    return hotel_keys

In [96]:
hotel_keys = get_hotel_keys()
len(hotel_keys)

416

### ------------xpath 확인 -> df확인 --------------

In [97]:
links = []
url = "https://www.yanolja.com/hotel/"
for hotel_key in hotel_keys:
    hotel_url = url + hotel_key
    links.append(hotel_url)
len(links)

416

In [98]:
# link 확인
link = links[0]
headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"}
req = requests.get(link, headers=headers)
response = TextResponse(req.url, body=req.text, encoding="utf-8")
response

<200 https://www.yanolja.com/hotel/3013410>

In [11]:
# xpath 확인
import json
location = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
location = json.loads(location)["props"]["pageProps"]["placeInfo"]["address"]
name = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/h1/text()')[0].extract()
level = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/div[1]/div[1]/text()')[0].extract()
score = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/div/span[1]/text()')[0].extract()
review_count = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/span/b/text()')[0].extract()
if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract() == []:
    room_type = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
else:    
    room_type = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract()
if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract() == []:
    room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
else:
    room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract()
price = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[4]/div/div[5]/span/text()').extract()
prices=[]
for num in range(len(room_type)):
    price = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
    price = json.loads(price)["props"]["pageProps"]["placeInfo"]["rooms"][num]["prices"][0]["price"]["discount"]["price"]
    prices.append(price)
        
location, name, level, score, review_count, room_type, room_detail, prices

('서울특별시 영등포구 경인로 870',
 '페어필드 바이 메리어트 서울',
 '4성급',
 '4.7',
 '8,797',
 ['Premier Twin',
  'Premier King',
  'Superior Single',
  'Premier Twin',
  'Premier Twin',
  'Premier King',
  'Premier King',
  'Premier Twin',
  'Premier King',
  'Premier Twin',
  'Premier King',
  'Premier Twin',
  'Premier King',
  'Premier Twin',
  'Premier King'],
 ['특가야놀자☆After 21pm Check-in/연박불가',
  '특가야놀자☆After 21pm Check-in/연박불가',
  '룸온리',
  '특가야놀자☆선착순 고층 룸UP+27시간스테이',
  '특가야놀자☆선착순 고층 룸UP',
  '특가야놀자☆선착순 고층 룸UP+25시간',
  '특가야놀자☆선착순 고층 룸UP+27시간',
  '특가야놀자☆선착순 고층 룸UP+25시간',
  '특가야놀자☆27시간스테이',
  '특가야놀자☆27시간스테이',
  '특가야놀자☆선착순 고층 룸UP',
  '특가야놀자☆Room and Parking Package',
  '특가야놀자☆Room and Parking Package',
  'stay for Breakfast',
  'stay for Breakfast'],
 ['59,900원',
  '59,900원',
  '60,000원',
  '63,000원',
  '63,000원',
  '63,000원',
  '63,000원',
  '63,000원',
  '63,000원',
  '63,000원',
  '63,000원',
  '71,800원',
  '71,800원',
  '77,300원',
  '77,300원'])

'0원'

In [147]:
# df 확인
locations, names, levels, scores, review_counts, room_types, room_details, prices_1, loading_urls = [], [], [], [], [] ,[], [], [], []

num1 = 0
for link in links[:10]:
    try:
        headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"}
        req = requests.get(link, headers=headers)
        response = TextResponse(req.url, body=req.text, encoding="utf-8")

        location = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
        location = json.loads(location)["props"]["pageProps"]["placeInfo"]["address"]
        name = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/h1/text()')[0].extract()
        level = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/div[1]/div[1]/text()')[0].extract()
        score = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/div/span[1]/text()')[0].extract()
        review_count = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/span/b/text()')[0].extract()
        if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract() == []:
            room_type = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
        else:    
            room_type = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract()
        if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract() == []:
            room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
        else:
            room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract()
        try:
            price = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[4]/div/div[5]/span/text()').extract()
            prices=[]
            for num in range(len(room_type)):
                price = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
                price = json.loads(price)["props"]["pageProps"]["placeInfo"]["rooms"][num]["prices"][0]["price"]["discount"]["price"]
                prices.append(price)
        except:
            prices= ["예약마감"]
    except:
        location = "페이지 없음"
        location = "페이지 없음"
        name = "페이지 없음"
        level = "페이지 없음"
        score = "페이지 없음"
        review_count = "페이지 없음"
        room_type = ["페이지 없음"]
        room_detail = ["페이지 없음"]
        price = ["페이지 없음"]
        prices= ["페이지 없음"]
        
    location = [location] * len(room_type)
    name = [name] * len(room_type)
    level = [level] * len(room_type)
    score = [score] * len(room_type)
    review_count = [review_count] * len(room_type)
    link = [link] * len(room_type)
    
    locations.extend(location)
    names.extend(name)
    levels.extend(level)
    scores.extend(score)
    review_counts.extend(review_count)
    room_types.extend(room_type)
    room_details.extend(room_detail)
    prices_1.extend(prices)
    loading_urls.extend(link)
    num1+=1
    print(str(num1) ,name[0])

1 페어필드 바이 메리어트 서울
2 나인트리 프리미어 명동2
3 영등포 GMS 호텔
4 신라스테이 구로
5 글래드 마포
6 노보텔 앰배서더 서울 용산
7 신라스테이 마포
8 더 파크호텔 서울
9 홀리데이인 익스프레스 서울 홍대
10 호텔 더 디자이너스 여의도


In [21]:
df = pd.DataFrame({
    'Name': names, 'Level': levels, 'Score':scores, 'Location': locations,
    'RoomType': room_types,'Roomdetail': room_details ,'Price': prices_1, 'Link': loading_urls, 'Review' : review_counts,
})
df.tail()

,Name,Level,Score,Location,RoomType,Roomdetail,Price,Link,Review
203,더 파크호텔 서울,4성급,4.6,서울특별시 영등포구 양평로 136,파크 홈 스위트,"마스크팩 무료 증정, 대중교통 이용시 커피 무료 이용권 제공","150,000원",https://www.yanolja.com/hotel/3016365,"1,478"
204,더 파크호텔 서울,4성급,4.6,서울특별시 영등포구 양평로 136,프리미어 패밀리 스위트,"마스크팩 무료 증정, 대중교통 이용시 커피 무료 이용권 제공","150,000원",https://www.yanolja.com/hotel/3016365,"1,478"
205,더 파크호텔 서울,4성급,4.6,서울특별시 영등포구 양평로 136,파크 로얄 스위트,"마스크팩 무료 증정, 대중교통 이용시 커피 무료 이용권 제공","270,000원",https://www.yanolja.com/hotel/3016365,"1,478"
206,더 파크호텔 서울,4성급,4.6,서울특별시 영등포구 양평로 136,패밀리 디럭스 트리플+욕조,"마스크팩 무료 증정, 대중교통 이용시 커피 무료 이용권 제공",0원,https://www.yanolja.com/hotel/3016365,"1,478"
207,더 파크호텔 서울,4성급,4.6,서울특별시 영등포구 양평로 136,파크 듀얼 스위트,"마스크팩 무료 증정, 대중교통 이용시 커피 무료 이용권 제공",0원,https://www.yanolja.com/hotel/3016365,"1,478"


### --------확인 끝 spider.py로 확장---------

In [1]:
# 3. items.py

In [73]:
%%writefile yanolja_hotel/yanolja_hotel/items.py
import scrapy

class YanoljaHotelItem(scrapy.Item):
    location = scrapy.Field()
    name = scrapy.Field()
    level = scrapy.Field()
    score = scrapy.Field()
    review_count = scrapy.Field()
    link = scrapy.Field()
    room_type = scrapy.Field()
    price = scrapy.Field()
    platform = scrapy.Field()
    date = scrapy.Field()

Overwriting yanolja_hotel/yanolja_hotel/items.py


In [2]:
# 4. spider.py

In [144]:
%%writefile yanolja_hotel/yanolja_hotel/spiders/spider.py
import scrapy
from yanolja_hotel.items import YanoljaHotelItem
from yanolja import get_hotel_keys
import json
import datetime

class Spider(scrapy.Spider):
    name = "YanoljaHotel"
    allow_domain = ["https://www.yanolja.com"]
    start_urls = ["https://www.yanolja.com/hotel/"]
    
    def parse(self, response):
        hotel_keys = get_hotel_keys()
        start_urls = "https://www.yanolja.com/hotel/"
        links = []
        headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"}
        for hotel_key in hotel_keys:
            hotel_url = start_urls + hotel_key
            links.append(hotel_url)
        for link in links:
            yield scrapy.Request(link, headers=headers, callback=self.parse_content)
    
    def parse_content(self, response):
        item = YanoljaHotelItem()
        if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract() == []:
            room_types = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
        else:    
            room_types = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract()
        if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract() == []:
            room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
        else:
            room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract()
        
        room_type_all = list(zip(room_types, room_detail))
        roomtypes = []
        for idx in range(len(room_type_all)):
            room_type_all[idx] = ",".join(room_type_all[idx])
            roomtypes.append(room_type_all[idx])   

        prices=[]
        for num in range(len(roomtypes)):
            price = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
            price = json.loads(price)["props"]["pageProps"]["placeInfo"]["rooms"][num]["prices"][0]["price"]["discount"]["price"]
            price = int(price.strip("원").replace(",",""))
            prices.append(price)

        locations = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
        locations = json.loads(locations)["props"]["pageProps"]["placeInfo"]["address"]
        names = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/h1/text()')[0].extract()
        levels = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/div[1]/div[1]/text()')[0].extract()
        scores = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/div/span[1]/text()')[0].extract()
        review_counts = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/span/b/text()')[0].extract()
        links = response.url
        platforms = response.url.split(".")[1]
        today = datetime.date.today()
        today = str(today)
        
        for i in range(len(roomtypes)):
            
            item["platform"] = platforms
            item["location"] = locations
            item["name"] = names
            item["level"] = levels
            item["score"] = scores
            item["review_count"] = review_counts
            item["link"] = links
            item["date"] = today
            item["room_type"] = roomtypes[i]
            item["price"] = prices[i]
            
            yield item

Overwriting yanolja_hotel/yanolja_hotel/spiders/spider.py


In [4]:
# 5. settings.py : robots.txt 여부 파악 

In [5]:
# 6. 프로젝트 실행

In [36]:
%pwd

'/home/ubuntu/python3/notebook/project/crawling/crawling-repo-2/in_progress'

In [145]:
%%writefile run.sh
cd ~/python3/notebook/project/crawling/crawling-repo-2/in_progress/yanolja_hotel
rm yanolja_hotel.csv
scrapy crawl YanoljaHotel -o yanolja_hotel.csv

Overwriting run.sh


In [ ]:
!/bin/bash run.sh

In [78]:
import pandas as pd
data = pd.read_csv('./yanolja_hotel/yanolja_hotel.csv')

In [79]:
data

,date,level,link,location,name,platform,price,review_count,room_type,score
0,2021-03-11,3성급,https://www.yanolja.com/hotel/3011807,서울특별시 금천구 벚꽃로56길 190,호텔해담채가산,yanolja,"48,000원","2,009","헐리우드더블,숙박불가★반나절호캉스 10시간 투숙, 오전9시체크인-오후11시체크아웃",4.7
1,2021-03-11,3성급,https://www.yanolja.com/hotel/3011807,서울특별시 금천구 벚꽃로56길 190,호텔해담채가산,yanolja,"57,500원","2,009","헐리우드 더블,특가야놀자☆18시체크인-연박불가, 테이블없음",4.7
2,2021-03-11,3성급,https://www.yanolja.com/hotel/3011807,서울특별시 금천구 벚꽃로56길 190,호텔해담채가산,yanolja,"65,000원","2,009","Deluxe Double,더블1개",4.7
3,2021-03-11,3성급,https://www.yanolja.com/hotel/3011807,서울특별시 금천구 벚꽃로56길 190,호텔해담채가산,yanolja,"65,000원","2,009","Deluxe Twin,싱글2개",4.7
4,2021-03-11,3성급,https://www.yanolja.com/hotel/3011807,서울특별시 금천구 벚꽃로56길 190,호텔해담채가산,yanolja,"65,500원","2,009","헐리우드 더블,특가야놀자☆27시간스테이+마스크팩 2매, 테이블없음",4.7
5,2021-03-11,3성급,https://www.yanolja.com/hotel/3011807,서울특별시 금천구 벚꽃로56길 190,호텔해담채가산,yanolja,"70,000원","2,009","헐리우드 더블,24시간STAY아무시간PKG, 테이블없음",4.7
6,2021-03-11,3성급,https://www.yanolja.com/hotel/3011807,서울특별시 금천구 벚꽃로56길 190,호텔해담채가산,yanolja,"70,000원","2,009","Deluxe Double,넷플릭스로 즐기는 호캉스 PKG, 더블1개",4.7
7,2021-03-11,3성급,https://www.yanolja.com/hotel/3011807,서울특별시 금천구 벚꽃로56길 190,호텔해담채가산,yanolja,"73,000원","2,009","Deluxe Double,해담도시락PKG, 더블1개",4.7
8,2021-03-11,3성급,https://www.yanolja.com/hotel/3011807,서울특별시 금천구 벚꽃로56길 190,호텔해담채가산,yanolja,"75,000원","2,009","Deluxe Double,해담채 올인원 1인 PKG",4.7
9,2021-03-11,4성급,https://www.yanolja.com/hotel/3015395,서울특별시 용산구 청파로20길 95,이비스 스타일 앰배서더 서울 용산,yanolja,"113,850원","2,141","슈페리어 트윈,★핫딜★이비스→노보텔, 취소불가",4.7


In [90]:
# 7. pipeline.py : 크롤링한 데이터를 mongodb 저장

In [ ]:
# scrapy 프로젝트에 pymongo 모듈 생성

In [1]:
%%writefile yanolja_hotel/yanolja_hotel/mongodb.py
import pymongo

client = pymongo.MongoClient("mongodb://15.165.254.37:27017")
collection = client.hotel_mk.hotel

Overwriting yanolja_hotel/yanolja_hotel/mongodb.py


In [ ]:
# pipeline.py 설정

In [10]:
%%writefile yanolja_hotel/yanolja_hotel/pipelines.py
from itemadapter import ItemAdapter
from .mongodb import collection

class YanoljaHotelPipeline:
    def process_item(self, item, spider):
        data = {"date": item["date"], "platform": item["platform"], "name": item["name"],
                "level": item["level"], "score": item["score"], "review": item["review_count"],
                "location": item["location"], "room_type": item["room_type"],
                "price": item["price"],"link": item["link"]}
        collection.insert(data)
        return item

Overwriting yanolja_hotel/yanolja_hotel/pipelines.py


In [89]:
# 8. settings.py -> pipeline 설정 변경

```
ITEM_PIPELINES = {
   'yanolja_hotel.pipelines.YanoljaHotelPipeline': 300,
}
```

In [ ]:
# 9. 프로젝트 실행!

In [ ]:
!/bin/bash run.sh

In [91]:
# 10.crontab 실행